# Install HuggingFace dependencies

In [1]:
!pip install transformers
!pip install tqdm

     |████████████████████████████████| 2.1MB 9.7MB/s 
     |████████████████████████████████| 901kB 37.2MB/s 
     |████████████████████████████████| 3.3MB 41.8MB/s 


# Mount Drive and Imports

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys, os, json

In [4]:
root_folder = "/content/drive/My Drive/cs182_project/"
output_dir = os.path.join(root_folder, "./model_save/")
data_path = os.path.join(root_folder, "yelp_review_training_dataset.jsonl")

In [5]:
sys.path.append(root_folder)

# Load Finetuned GPT-2 Model

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

from tqdm import tqdm

In [7]:
config = GPT2Config.from_pretrained(output_dir)
model = GPT2LMHeadModel.from_pretrained(output_dir)
tokenizer = GPT2Tokenizer.from_pretrained(output_dir)

In [8]:
if torch.cuda.is_available():
  device = "cuda"
  model.cuda()
else:
  device = "cpu"
model.to(device)
print(device)

cuda


# Do a sample review generation

In [9]:
model.eval()

prompt = "<|startoftext|> OMG I absolutely loved"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)
model = model.to(device)

print(generated)

sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=50, 
                                max_length = 300,
                                top_p=0.95, 
                                num_return_sequences=3
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[50257, 43821,   314,  5543,  6151]], device='cuda:0')
0:  OMG I absolutely loved this place. Amazing little cafe and I didn't realize there was a lot to choose from since that is what really got to mind during the week. Food was good and we had dessert. We were both impressed and the service was very helpful and pleasant. The place is well equipped with great music on the TVs. I had the red velvet pancakes and chocolates for the dessert for dessert and it was so good. The icing dessert. The cupcakes were perfectly proportioned. I could make it to my girlfriend. The service too many people in line.


1:  OMG I absolutely loved this place! They have good deals and they really take great care of their customers! They have so many different dresses for men. If I could give this place zero stars they would.


2:  OMG I absolutely loved this place. The menu is quite limited, but the food is so good, that is not my main preference. I found the churros here to be delicious, and had a 

In [10]:
from data_parsing import load_dataset

In [11]:
data = load_dataset(data_path)

In [12]:
reviews234 = [i for i in data if i[1] == 2 or i[1] == 3 or i[1] == 4]

In [13]:
len(reviews234)

142543

In [14]:
tokenizer.decode(sample_outputs[0], skip_special_tokens=True)

" OMG I absolutely loved this place. Amazing little cafe and I didn't realize there was a lot to choose from since that is what really got to mind during the week. Food was good and we had dessert. We were both impressed and the service was very helpful and pleasant. The place is well equipped with great music on the TVs. I had the red velvet pancakes and chocolates for the dessert for dessert and it was so good. The icing dessert. The cupcakes were perfectly proportioned. I could make it to my girlfriend. The service too many people in line."

In [18]:
new_data = []

In [ ]:
for i in tqdm(range(0, 5000)):
  text, review = reviews234[i]
  prompt = "<|startoftext|>" + " " + ' '.join(text.split()[:15])
  generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
  generated = generated.to(device)
  model = model.to(device)

  sample_outputs = model.generate(
                                  generated, 
                                  #bos_token_id=random.randint(1,30000),
                                  do_sample=True,   
                                  top_k=50, 
                                  max_length = 300,
                                  top_p=0.95, 
                                  num_return_sequences=1
                                  )
  
  gpt_output = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
  new_data.append((gpt_output, review))


  1%|          | 52/5000 [01:15<1:49:12,  1.32s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
  2%|▏         | 83/5000 [02:04<2:51:03,  2.09s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
  4%|▎         | 180/5000 [04:15<1:15:40,  1.06it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
  4%|▎         | 182/5000 [04:15<58:46,  1.37it/s]  Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
  4%|▍         | 195/5000 [04:30<1:37:16,  1.21s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
  5%|▍         | 240/5

In [ ]:
new_data[10], data[10]

((' Like walking back in time, every Saturday morning my sister and I have tried the menu and always loved it.We will go again when we need something quick.',
  4.0),
 ('Like walking back in time, every Saturday morning my sister and I was in a bowling league and after we were done, we\'d spend a few quarters playing the pin ball machines until our mother came to pick us up.\n\nMy sister was daring and play the machines hard, she was afraid of that "tilt" showing up and freezing the game.  I, on the other hand was a bit more gentler and wanted to make sure I got my quarter\'s worth.\n\nThis place has rows and rows of machines, some are really old and some are more of a mid 80\'s theme.  There is even a Ms pac man!  It was fun to spend an afternoon playing the machines and remembering all the fun of my early teen years.',
  4.0))

In [ ]:
str(new_data)

'[(\' Total bill for this horrible service? Over $8Gs. These crooks were a little rude and they said $7 per person would be out. The manager was on meth meth and no hello. I have a business complaint and he needs to be fired and screwed up in the front of the country.\', 1.0), (" I *adore* Travis at the Hard Rock\'s new Kelly Cardenas location and I had a fantastic experience. He\'s on time and friendly. They have complimentary wi-fi available to pick up beer, snacks, drink and even other items you can\'t find at their prices. I\'ve also been there more times for lunch and a drink than what it was at the time or cheaper. The St. O.J. (L*T. K*K) is a local brewery in the great to my god. I love the great place and the music. Best Dive Tea, I love the drinks!", 5.0), (" I have to say that this office really has it all. The doctors are wonderful and attentive. The front desk staff was so kind and sweet. They even gave the front desk to me. They answered all my questions before i got them 

In [ ]:
rs = json.dumps(dict(new_data))

In [ ]:
with open(os.path.join(root_folder, 'new_data3.json'), 'w') as f:
  json.dump(rs, f)